In [4]:
import numpy
from math import *

def activate_func(x, derivative=0):
    if derivative == 0:
        return 1.0 / (1 + np.exp(-x))
    else:
        return activate_func(x) * (1 - activate_func(x))
        
a = np.array([0, 2, 1])
b = np.array([a, a + 1, a + 2])
c = np.array([[2, 4], [4, 3], [4, 1]])
d = np.array([[0, 2, 1]])

print("a shape: ", a.shape)
print("a: ", a)
print("a.T shape:", a.T.shape)
print("a.T: ", a.T)
print("b is: ",b)
print("b * b is: ", b * b)
print("a * b is: ", a * b)
print("a length: ", a.size)
print("c shape: ", c.shape)
print(type(c))
print("c size: ", c.size)
print("d shape: ", d.shape)
print("d: ", d)
print("d.T shape: ", d.T.shape)
print("d.T: ", d.T)
print("d - d.T", d - d.T)
print('d[0]: ', d[0])
print("activate d:", activate_func(d))
print('d*d: ', d * d)
print('d.dot(c): ', d.dot(c))
print('c0: ', c[0])
for i in c:
    print(i)
    print(type(i))
    print(isinstance(i, numpy.ndarray))

a shape:  (3,)
a:  [0 2 1]
a.T shape: (3,)
a.T:  [0 2 1]
b is:  [[0 2 1]
 [1 3 2]
 [2 4 3]]
b * b is:  [[ 0  4  1]
 [ 1  9  4]
 [ 4 16  9]]
a * b is:  [[0 4 1]
 [0 6 2]
 [0 8 3]]
a length:  3
c shape:  (3, 2)
<class 'numpy.ndarray'>
c size:  6
d shape:  (1, 3)
d:  [[0 2 1]]
d.T shape:  (3, 1)
d.T:  [[0]
 [2]
 [1]]
d - d.T [[ 0  2  1]
 [-2  0 -1]
 [-1  1  0]]
d[0]:  [0 2 1]
activate d: [[ 0.5         0.88079708  0.73105858]]
d*d:  [[0 4 1]]
d.dot(c):  [[12  7]]
c0:  [2 4]
[2 4]
<class 'numpy.ndarray'>
True
[4 3]
<class 'numpy.ndarray'>
True
[4 1]
<class 'numpy.ndarray'>
True


In [2]:
###
#  @author xxx
#  created on 2016.6.29
###
import numpy as np
from math import *


class BPNN():
    
    ##  X in form of:
    ##
    ##[[0, 0, 0],
    ## [0, 0, 0], 
    ## ...
    ## [0, 0, 0]]
    
    ##  Y in form of:
    ##
    ##[[0],
    ## [0], 
    ## ...
    ## [0]]
    
    
    def __init__(self, X, Y):
        
        self.X = X
        self.Y = Y 
        self.eta1 = 0.1
        self.eta2 = 0.1
        self.test_count = X.shape[0]
        self.feature_count = X.shape[-1]
        self.output_count = Y.shape[1]
        #it is said that sqrt(output node number + input node number) + 5 is a feasible
        self.h_node_number = ceil(sqrt(self.output_count + self.feature_count) + 5)
        np.random.seed(0)
        self.W1 = 2 * np.random.random_sample((self.feature_count, self.h_node_number)) - 1
        self.W2 = 2 * np.random.random_sample((self.h_node_number, self.output_count)) - 1
        print("test case size: %d, X node number: %d, \
hidden layer node number: %d, Y node number: %d"%(self.test_count, \
              self.feature_count, self.h_node_number, self.output_count))
        
        #self.normalize()
        
    
    def normalize(self):
        
        x_feature_bounds = list(map(lambda x: (np.max(x), np.min(x)), self.X.T))
        #y_bounds = list(map(lambda y: (np.max(y), np.min(y)), self.Y.T))
        
        NX = [list(map(lambda x, y: (x - y[1])/ (y[0] - y[1]), testcase, x_feature_bounds)) for testcase in self.X]
        NX = np.array(NX)
        self.X = NX
        
        #NY = [list(map(lambda x, y: (x - y[1])/ (y[0] - y[1]), testcase, y_bounds)) for testcase in self.Y]
        #NY = np.array(NY)
        #self.Y = NY
        
    
    def activate_func(self, x, derivative=0):
        
        if derivative == 0:            
            #be careful!
            #use np.exp instaad of math.exp to enable array-like data computing
            return 1.0 / (1 + np.exp(-x))
        else:
            return self.activate_func(x) * (1 - self.activate_func(x))
        
    
    def train(self, train_time=100, precision=0.001):
        
        for i in range(train_time):
            test_case_id = i % self.Y.shape[0]
            
            input_x = np.array([self.X[test_case_id].tolist()])
            input_y = np.array([self.Y[test_case_id].tolist()])
            
            #get value of hidden node
            net1 = np.dot(input_x, self.W1)
            #output value of hidden node after activated(normalized)
            hidden_node = self.activate_func(net1)
            
            #get value of output node
            net2 = np.dot(hidden_node, self.W2)
            #output value of output node after activated(normalized)
            trained_y = self.activate_func(net2)
            
            error_y = trained_y - input_y
            if i%5000==0:
                print(error_y)
                print(self.X[test_case_id])
                print(self.Y[test_case_id])
           
            delta_w2 = error_y * self.activate_func(net2, 1)
            adjust_w2 = - self.eta2 * np.dot(hidden_node.T, delta_w2)
            
            delta_w1 = np.dot(self.W2, delta_w2.T) * self.activate_func(net1, 1).T
            adjust_w1 = -self.eta1 * np.dot(delta_w1, input_x).T
            
            self.W1 += adjust_w1
            self.W2 += adjust_w2
       
        print('W1:', self.W1)
        print('\n')
        print('W2:', self.W2)
        print('\n')
        
        
    def predict(self, x):
        
        res = []
        for case in x:
            case = np.array(case.tolist())
            net1 = case.dot(self.W1)
            hidden_layer = self.activate_func(net1)
        
            net2 = hidden_layer.dot(self.W2)
            y = self.activate_func(net2)
            y = y.tolist()
            res.append(y)
            
        return np.array(res)        
    
    
if __name__ == '__main__':
    
    X = np.array([[2, 2, 2, 2, 2],
            [0, 3, 6, 1, 7],
            [1, 2, 1, 9, 4],
            [1, 1, 1, 1, 1],
            [4, 5, 7, 2, 7], 
            [4, 4, 4, 4, 4],
            [2, 2, 2, 2, 2],
            [3, 2, 7, 3, 6],
            [3, 3, 2, 6, 1]])
    
    XX = np.array([[0, 0], 
                  [0, 1], 
                  [1, 1], 
                  [ 1, 0]])
    
    Y = np.array([[0, 1], 
                 [1, 2], 
                 [1, 1], 
                 [0, 3], 
                 [1, 4],
                 [0, 4],
                 [0, 2], 
                 [1, 0],
                 [1, 2]])
    
    YY = np.array([[0], 
                  [1], 
                  [0], 
                  [1]])
    
    bpnn = BPNN(X, Y)
    
    bpnn.train(100000)
    
    print(X)
    print(Y)
    
    y = bpnn.predict(X)
    print(y)

test case size: 9, X node number: 5, hidden layer node number: 8, Y node number: 2
[[ 0.33659454 -0.933003  ]]
[2 2 2 2 2]
[0 1]
[[ 0.10410535 -3.00019671]]
[4 4 4 4 4]
[0 4]
[[-0.04274907 -1.00009403]]
[0 3 6 1 7]
[1 2]
[[ 0.04457608 -1.00009619]]
[2 2 2 2 2]
[0 2]
[[-0.01600541 -0.00056719]]
[1 2 1 9 4]
[1 1]
[[-0.02326134  0.99996599]]
[3 2 7 3 6]
[1 0]
[[ 0.053411   -2.00013306]]
[1 1 1 1 1]
[0 3]
[[-0.04519641 -1.00004953]]
[3 3 2 6 1]
[1 2]
[[-0.03623204 -3.00002235]]
[4 5 7 2 7]
[1 4]
[[  2.30035621e-02  -3.52602172e-05]]
[2 2 2 2 2]
[0 1]
[[ 0.02252867 -3.00002143]]
[4 4 4 4 4]
[0 4]
[[-0.01360732 -1.00001411]]
[0 3 6 1 7]
[1 2]
[[ 0.01953373 -1.00002676]]
[2 2 2 2 2]
[0 2]
[[-0.00679268 -0.00018555]]
[1 2 1 9 4]
[1 1]
[[-0.01256526  0.99998899]]
[3 2 7 3 6]
[1 0]
[[ 0.03141034 -2.00005571]]
[1 1 1 1 1]
[0 3]
[[-0.02771436 -1.00002213]]
[3 3 2 6 1]
[1 2]
[[-0.02366705 -3.00000935]]
[4 5 7 2 7]
[1 4]
[[  1.56431100e-02  -1.80120062e-05]]
[2 2 2 2 2]
[0 1]
[[ 0.01501788 -3.000011

In [5]:
from pybrain.tools.shortcuts import buildNetwork
from pybrain.datasets import SupervisedDataSet
from pybrain.supervised.trainers import BackpropTrainer

ds = SupervisedDataSet(5, 1)
ds.addSample((2, 3, 5, 6, 9),(0,))
ds.addSample((0, 3, 6, 1, 7),(1,))
ds.addSample((1, 2, 1, 9, 4),(1,))
ds.addSample((8, 1, 1, 5, 2),(0,))
ds.addSample((4, 5, 7, 2, 7),(1,)) 
ds.addSample((8, 4, 4, 7, 3),(0,))
ds.addSample((2, 2, 1, 4, 3),(0,))
ds.addSample((3, 2, 7, 3, 6),(1,))
ds.addSample((3, 3, 2, 6, 1),(1,))

net = buildNetwork(5, 7, 1)
trainer = BackpropTrainer(net, ds)

trainer.trainUntilConvergence()

([0.32216640103664312,
  0.23724558179019239,
  0.1912103515455689,
  0.16273703897333386,
  0.14391939161790804,
  0.13114676772544198],
 [0.34725150041660724,
  0.21161183384224591,
  0.15274677771795689,
  0.11645648330006249,
  0.098563772131663188,
  0.087623289644630994,
  0.086381344055837028])

In [14]:
def normalize(NX, NY):
        
        x_feature_bounds = list(map(lambda x: (np.max(x), np.min(x)), X.T))
        print(x_feature_bounds)
        y_bounds = list(map(lambda y: (np.max(y), np.min(y)), Y.T))
        print(y_bounds)
        
        NX = [list(map(lambda x, y: (x - y[1])/ (y[0] - y[1]), testcase, x_feature_bounds)) for testcase in X]
        NX = np.array(NX)
        
        NY = [list(map(lambda x, y: (x - y[1])/ (y[0] - y[1]), testcase, y_bounds)) for testcase in Y]
        NY = np.array(NY)
        
        print(NX)
        print('')
        print(NY)
        
X = np.array([[2, 2, 2, 2, 2],
            [0, 3, 6, 1, 7],
            [1, 2, 1, 9, 4],
            [1, 1, 1, 1, 1],
            [4, 5, 7, 2, 7], 
            [4, 4, 4, 4, 4],
            [2, 2, 2, 2, 2],
            [3, 2, 7, 3, 6],
            [3, 3, 2, 6, 1]])
    
Y = np.array([[0, 1], 
                 [1, 2], 
                 [1, 1], 
                 [0, 3], 
                 [1, 4],
                 [0, 4],
                 [0, 2], 
                 [1, 0],
                 [1, 2]])

normalize(X, Y)

[(4, 0), (5, 1), (7, 1), (9, 1), (7, 1)]
[(1, 0), (4, 0)]
[[ 0.5         0.25        0.16666667  0.125       0.16666667]
 [ 0.          0.5         0.83333333  0.          1.        ]
 [ 0.25        0.25        0.          1.          0.5       ]
 [ 0.25        0.          0.          0.          0.        ]
 [ 1.          1.          1.          0.125       1.        ]
 [ 1.          0.75        0.5         0.375       0.5       ]
 [ 0.5         0.25        0.16666667  0.125       0.16666667]
 [ 0.75        0.25        1.          0.25        0.83333333]
 [ 0.75        0.5         0.16666667  0.625       0.        ]]

[[ 0.    0.25]
 [ 1.    0.5 ]
 [ 1.    0.25]
 [ 0.    0.75]
 [ 1.    1.  ]
 [ 0.    1.  ]
 [ 0.    0.5 ]
 [ 1.    0.  ]
 [ 1.    0.5 ]]


In [4]:
arr = np.array([[2, 3, 5], [3, 3, 3]])
brr = np.array([[3, 3, 4]])
arr + brr

array([[5, 6, 9],
       [6, 6, 7]])